# 종목 정보 최신화

In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 1. 종목 최신화
print(">> 1. 시가총액 5천억 이상 ticker 최신화")

# 1.0. 티커 조회
print("1.0. 티커 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 1.1. 업데이트 기간 설정
print("1.1. 업데이트 기간 설정")
date = str(datetime.now().date()).replace('-', '')
print("기간: (%s)" %(date))

# 1.2. 기존 데이터 제거
print("1.2. 기존 데이터 제거 (%s)" %(date))
cursor.execute("DELETE FROM stocks_price WHERE date = " + date)
db.commit()
db.close()

# 1.3. 새로운 데이터 업데이트
print("1.3. 새로운 데이터 업데이트 (%s)" %(date))
for ticker in tickers:
  try:
    df = stock.get_market_ohlcv_by_date(str(datetime.now().date() - timedelta(days=1)).replace('-', ''), date, ticker)
    time.sleep(0.1)
    df['ticker'] = ticker
    df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
    df.index.names = ['date']
    df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, ticker)
    pass

>> 1. 시가총액 5천억 이상 ticker 최신화
1.0. 티커 조회
1.1. 업데이트 기간 설정
기간: (20211103)
1.2. 기존 데이터 제거 (20211103)
1.3. 새로운 데이터 업데이트 (20211103)


# 볼린저 밴드 최신화

In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 2. 볼린저밴드 최신화
print(">> 2. 볼린저 밴드 최신화")

# 2.0. 티커 조회
print("2.0. 티커 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 2.1. 업데이트 기간 설정
print("2.1. 업데이트 기간 설정")
date = str(datetime.now().date()).replace('-', '')
print("기간: (%s)" %(date))

# 2.2. 기존 데이터 제거
print("2.2. 기존 데이터 제거 (%s)" %(date))
cursor.execute("DELETE FROM boll WHERE date =" + date)
db.commit()

# 2.3. 새로운 데이터 업데이트
print("2.3. 새로운 데이터 업데이트 (%s)" %(date))
day = 20
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' AND date <= '" + date + "' ORDER BY date DESC LIMIT " + str(day + 1))
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['period'] = day
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = round(avg - std * 2)
      me = round(avg)
      hi = round(avg + std * 2)
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    bollDf[:1].to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1)
    pass

# MFI 최신화

In [8]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 3. MFI 최신화
print(">> 3. MFI 최신화")

# 3.0. 티커 조회
print("3.0. 티커 조회")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 3.1. 업데이트 기간 설정
print("3.1. 업데이트 기간 설정")
date = str(datetime.now().date()).replace('-', '')
print("기간: (%s)" %(date))

# 3.2. 기존 데이터 제거
print("3.2. 기존 데이터 제거 (%s)" %(date))
cursor.execute("DELETE FROM mfi WHERE date =" + date)
db.commit()

# 3.3. 새로운 데이터 업데이트
print("3.3. 새로운 데이터 업데이트 (%s)" %(date))
day = 10
for ticker in tickers:
  print(ticker)
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' AND date <= '" + date + "' ORDER BY date DESC LIMIT " + str(day + 2))
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
    # Set Dataframe
    priceDf['period'] = day
    priceDf['tp'] = 0
    priceDf['mfi'] = 0.0
    priceDf['mfi_diff'] = 0.0
    # Set typical price
    for idx in range(0, len(priceDf)):
      row = priceDf.iloc[idx]
      priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
    # Set MFI
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      positiveRMF = 0
      negativeRMF = 0          
      for i in range(idx, idx + day):
        today = priceDf.iloc[i]
        yesterday = priceDf.iloc[i + 1]
        if today['tp'] > yesterday['tp']:
          positiveRMF += today['tp'] * today['volume']
        elif today['tp'] < yesterday['tp']:
          negativeRMF += today['tp'] * today['volume']
      MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
      priceDf.at[idx, 'mfi'] = MFI
      if idx > 0:
        priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
    mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    mfiDf[:1].to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
    db.commit()
  except Exception as ex1:
    print('ex1', ex1, ticker)
    pass
db.close()

>> 3. MFI 최신화
3.0. 티커 조회
3.1. 업데이트 기간 설정
기간: (20211105)
3.2. 기존 데이터 제거 (20211105)
3.3. 새로운 데이터 업데이트 (20211105)
000060
000070
000080
000100
000120
000150
000210
000240
000270
000370
000400
000640
000660
000670
000720
000810
000815
000880
000990
001040
001120
001230
001390
001430
001440
001450
001530
001680
001720
001740
001800
001820
001880
002240
002350
002380
002790
002840
003000
003070
003090
003230
003240
003410
003470
003490
003520
003530
003540
003550
003580
003670
003690
003850
004000
004020
004170
004370
004490
004800
004990
005070
005090
005180
005250
005300
005380
005385
005387
005390
005440
005490
005610
005690
005830
005850
005880
005930
005935
005940
006040
006120
006260
006280
006360
006390
006400
006405
006650
006800
00680K
007070
007310
007340
007570
007690
007700
008490
008560
008730
008770
008930
009150
009240
009420
009450
009540
009830
009900
009970
010050
010060
010120
010130
010620
010780
010950
011000
011070
011170
011200
011210
011780
011790
012330
012450
012510


In [6]:
mfiDf[:1]

,ticker,period,tp,mfi,mfi_diff
date,,,,,
2021-11-05,000060,10,29583,37.475381,0.563885
